In [2]:
from datetime import datetime, timedelta
from math import sin, cos, asin, radians, degrees
import pandas as pd

In [3]:
df = pd.read_csv("lat_lontable.csv")

In [4]:
# 計算太陽仰角的函數
def calculate_solar_altitude(lat, lon, date_time):
    # 地點經緯度轉換為弧度
    lat = radians(lat)
    lon = radians(lon)

    # 計算儒略日
    timezone_offset = 8  # 假設為台灣時區
    utc_time = date_time - timedelta(hours=timezone_offset)
    jd = 367 * utc_time.year - int((7 * (utc_time.year + int((utc_time.month + 9) / 12))) / 4) + int((275 * utc_time.month) / 9) + utc_time.day + 1721013.5 + (utc_time.hour + utc_time.minute / 60 + utc_time.second / 3600) / 24

    # 太陽的均數近點角與黃經
    M = radians(357.5291 + 0.98560028 * (jd - 2451545))
    L = radians(280.459 + 0.98564736 * (jd - 2451545) + (1.9148 * sin(M)) + (0.0200 * sin(2 * M)))

    # 赤緯
    epsilon = radians(23.44)  # 地軸傾角
    delta = asin(sin(epsilon) * sin(L))

    # 時角
    time_offset = (utc_time.hour + utc_time.minute / 60 + utc_time.second / 3600 + lon / radians(15))
    H = radians(15 * (time_offset - 12))

    # 仰角
    altitude = asin(sin(lat) * sin(delta) + cos(lat) * cos(delta) * cos(H))
    return degrees(altitude)

# 設定計算範圍
start_time = datetime(2024, 1, 1, 0, 0)
end_time = datetime(2024, 11, 1, 0, 0)
time_interval = timedelta(minutes=10)

# 計算每個地點的仰角
results = []
for _, row in df.iterrows():
    current_time = start_time
    while current_time < end_time:
        altitude = calculate_solar_altitude(row['緯度'], row['經度'], current_time)
        results.append({
            "location":row['location'],
            "經度":row['經度'],
            "緯度":row['緯度'],
            "地點名稱": row['地點名稱'],
            "DateTime": current_time,
            "方位角":row['方位角'],
            "太陽仰角": altitude
        })
        current_time += time_interval

# 將結果轉換為DataFrame
results_df = pd.DataFrame(results)



In [5]:
results_df

,location,經度,緯度,地點名稱,DateTime,方位角,太陽仰角
0,1,121.544444,23.899444,頂樓一號,2024-01-01 00:00:00,181.000,-88.366208
1,1,121.544444,23.899444,頂樓一號,2024-01-01 00:10:00,181.000,-86.202505
2,1,121.544444,23.899444,頂樓一號,2024-01-01 00:20:00,181.000,-83.943446
3,1,121.544444,23.899444,頂樓一號,2024-01-01 00:30:00,181.000,-81.666617
4,1,121.544444,23.899444,頂樓一號,2024-01-01 00:40:00,181.000,-79.383910
...,...,...,...,...,...,...,...
746635,17,121.600000,23.983333,自家倉庫,2024-10-31 23:10:00,185.125,-75.947259
746636,17,121.600000,23.983333,自家倉庫,2024-10-31 23:20:00,185.125,-77.567816
746637,17,121.600000,23.983333,自家倉庫,2024-10-31 23:30:00,185.125,-78.921182
746638,17,121.600000,23.983333,自家倉庫,2024-10-31 23:40:00,185.125,-79.899069


In [6]:
# 輸出為CSV檔案
results_df.to_csv("./solar_altitudes.csv", index=False)